In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using BenchmarkTools
using TransportMap
using QuadGK
using Distributions
using Random
using LoopVectorization
using Test
using SparseArrays
using AdaptiveTransportMap: vander, transform!, evaluate, ncoeff, optimize, negative_log_likelihood!, derivative
using Quadrature
using Cubature
using FiniteDiff

### Gradient and Hessian of the log pdf

In [3]:
Nx = 100
Ne = 300
m = 10

X = randn(Nx, Ne) .* randn(Nx, Ne) .+ randn(Nx)
M = HermiteMap(m, X)

@time optimize(M, X, "split"; withqr = true, P = serial)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:13


 33.585821 seconds (108.17 M allocations: 16.922 GiB, 4.75% gc time)


HermiteMap(10, 100, LinearTransform(100, [-1.9388115412536764, -0.5532840915970996, 0.6649510381692495, 0.8066244424946927, 1.4603598696781142, -1.4589681336803537, -0.9824037374192759, -0.20679245486456055, -1.1291131135026242, 0.6906625282794049  …  -0.37101560389918037, -0.7188384105130516, 1.9141484922491452, -0.41860810828244416, -1.6650320190314736, 1.410747568280138, -1.9536171293522333, 0.2517759861817628, -1.3866687836217055, 0.16619895667801496], [1.003333887605085 0.0 … 0.0 0.0; 0.0 0.889799322434419 … 0.0 0.0; … ; 0.0 0.0 … 1.0213865816432859 0.0; 0.0 0.0 … 0.0 0.9260049299342631], true), MapComponent[MapComponent(10, 10, 1, IntegratedFunction(10, 10, 1, Rectifier("softplus"), ParametricFunction(ExpandedFunction(10, 10, 1, MultiBasis(Basis of 10 functions: Constant -> 8th degree Probabilistic Hermite function
, 1), [1; 2; … ; 9; 10], [0.09173313339897951, -0.8336281241597296, -0.3076331206110271, -2.6560035858924618, -0.8156438246869363, -0.7097151201760736, -1.269929867275

In [4]:
@time log_pdf(M, X)

  1.121670 seconds (2.73 M allocations: 369.135 MiB, 3.77% gc time)


300-element Array{Float64,1}:
 -148.00842938794818
 -148.42110561322343
 -137.3183331795096
 -184.45411758023175
 -135.63502485205632
 -147.17804161472566
 -226.19485388547355
 -149.80243077765397
 -163.8619223889392
 -147.12756931017006
 -150.0183487398815
 -132.6672245817722
 -165.39203355434793
    ⋮
 -121.02190566359589
 -123.30367295517178
 -125.4585912401667
 -131.34454904158483
 -140.1284062462099
 -129.71073678265464
 -159.47748237981594
 -116.34194973867599
 -130.35756262042497
 -143.21345376700563
 -143.03348885470103
 -113.43343674323893

In [5]:
@time grad_x_log_pdf(M, X)

 11.846705 seconds (30.53 M allocations: 27.889 GiB, 11.09% gc time)


300×100 Array{Float64,2}:
  -0.0153203    2.42071    -3.7929    …  -0.34405       0.174084
  -0.76679     -1.00578    -3.66141      -0.33415       0.835249
  -3.43962      3.65164     2.3769        0.579707     -0.1413
   4.02974      6.35697    -1.0862        0.317252     -0.714295
   2.10512      1.8903      1.54091       0.182127     -1.16645
   2.3411      -0.36978    -4.57903   …   0.333406     -0.468025
  -4.04687    -46.6121    -22.4545        2.49388       0.592682
  -0.0846449    0.963411   -6.81571       1.26821       0.330123
 -12.0067     -22.3289      4.78444       0.246952     -0.714346
 -24.0613      12.3354     29.5016        0.000673324  -1.00073
  -0.873367    -0.136755    1.54854   …  -0.354042      0.115718
   0.884072     1.1132     -1.79355      -1.16399       0.648909
   0.933196    -2.15938     1.40781      -2.77057       0.0549741
   ⋮                                  ⋱                
   3.92888    -19.7817     12.095         0.362091     -0.69975
   0.462386 

In [9]:
@time d2logψ = hess_x_log_pdf(M.C[70], X[1:70,:])

  3.482114 seconds (8.95 M allocations: 15.758 GiB, 20.69% gc time)


300×70×70 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [11]:
d2logψ[1,:,:]

70×70 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0
 ⋮            

In [12]:
using Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278


In [8]:
@time optimize(M, X, nothing; withqr = true, P = serial)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


  0.321003 seconds (56.81 k allocations: 178.260 MiB, 18.10% gc time)


HermiteMap(10, 100, LinearTransform(100, [-1.9388115412536764, -0.5532840915970996, 0.6649510381692495, 0.8066244424946927, 1.4603598696781142, -1.4589681336803537, -0.9824037374192759, -0.20679245486456055, -1.1291131135026242, 0.6906625282794049  …  -0.37101560389918037, -0.7188384105130516, 1.9141484922491452, -0.41860810828244416, -1.6650320190314736, 1.410747568280138, -1.9536171293522333, 0.2517759861817628, -1.3866687836217055, 0.16619895667801496], [1.003333887605085 0.0 … 0.0 0.0; 0.0 0.889799322434419 … 0.0 0.0; … ; 0.0 0.0 … 1.0213865816432859 0.0; 0.0 0.0 … 0.0 0.9260049299342631], true), MapComponent[MapComponent(10, 10, 1, IntegratedFunction(10, 10, 1, Rectifier("softplus"), ParametricFunction(ExpandedFunction(10, 10, 1, MultiBasis(Basis of 10 functions: Constant -> 8th degree Probabilistic Hermite function
, 1), [1; 2; … ; 9; 10], [0.09564406871613584, -1.5235117095029267, 0.14859370117804918, -4.413286558881069, 0.6568369757672253, -3.796396743470617, 0.7444250305312987

In [ ]:
@time for i=1:Nx
    @show i
    hess_x_log_pdf(M.C[i], X[1:i,:])
end


In [ ]:
@time hess_x_log_pdf(M, X)

i = 1
i = 2
i = 3
i = 

### Integration

In [186]:
Nx = 100
Ny = 50
m = 20
Ne = 400
X = randn(Nx, Ne).*randn(Nx, Ne) #.+ 0.5*randn(Nx).*cos.(randn(Nx, Ne) .* randn(Nx, Ne))

# L = LinearTransform(X)
# transform!(L, X);
# C = MapComponent(m, Nx)
# @time C, _ = optimize(C, X, 10)

In [187]:
M = HermiteMap(m, X);

In [188]:
@time M_noqr = optimize(M, X, 10; P = serial, start = Ny+1);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:13


 19.530544 seconds (69.35 M allocations: 14.972 GiB, 7.45% gc time)


In [189]:
@time M_qr = optimize(M, X, 10; P = serial, withqr = true, start = Ny+1);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:10


 10.677751 seconds (44.03 M allocations: 13.718 GiB, 9.93% gc time)


In [31]:
function timing()
    M = HermiteMap(m, X)
    @btime begin 
        optimize($M, $X, $10; P = serial, withqr = false, start = Ny+1)
    end
    
    @btime begin 
        optimize($M, $X, $10; P = serial, withqr = true, start = Ny+1)
    end
end

timing (generic function with 1 method)

In [32]:
timing();

  11.257 s (42606958 allocations: 13.65 GiB)
  21.346 s (45813192 allocations: 21.12 GiB)


In [11]:
timing();

  14.597 s (42433511 allocations: 13.37 GiB)


In [29]:
timing();

  973.450 ms (1054425 allocations: 1.43 GiB)


In [14]:
methods(update_component)

# 1 method for generic function "update_component":
[1] update_component(C::MapComponent, X, reduced_margin::Array{Int64,2}, S::Storage) in AdaptiveTransportMap at /media/mat/HDD/AdaptiveTransportMap/src/mapcomponent/greedyfit.jl:249

In [30]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = thread)
    end
end

timing (generic function with 1 method)

In [88]:
timing();

  297.730 ms (1000725 allocations: 1.35 GiB)


### Test optimization with QR

In [45]:
Nx = 3
Ne = 8
m = 20

idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]


Nψ = 6
coeff = [ 0.20649582065364197;
         -0.5150990160472986;
          2.630096893080717;
          1.13653076177397;
          0.6725837371023421;
         -1.3126095306624133]
C = MapComponent(m, Nx, idx, coeff; α = 1e-6);

Ne = 100


# The QR decomposition is not unique!

X = randn(Nx, Ne) .* randn(Nx, Ne) + cos.(randn(Nx, Ne)) .* exp.(-randn(Nx, Ne).^2)

L = LinearTransform(X)
transform!(L, X)
S = Storage(C.I.f, X)
F = QRscaling(S)
newidx = [1 1 1]

Snew = update_storage(S, X, newidx)
Fupdated = updateQRscaling(F, Snew)

Fnew = QRscaling(Snew)

@test norm(Fupdated.D - Fnew.D)<1e-8
@test norm(Fupdated.Dinv - Fnew.Dinv)<1e-8

Test Passed